In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ephem
%matplotlib qt5

In [2]:
Tiangong = pd.read_csv('Tiangong.csv', parse_dates = [0], index_col = 0)
Tiangong

,Inclination,RAAN,Eccentricity,AoP,Mean Anomaly,Mean Motion,Interval
2011-09-29 17:23:49,42.7802,196.4777,0.010130,130.5581,230.4736,16.032707,0
2011-09-29 18:53:44,42.7775,196.0736,0.011559,131.6747,229.8960,16.009910,5395
2011-09-29 20:39:14,42.7837,195.6141,0.011007,132.4039,291.9162,16.009548,6330
2011-09-29 23:35:42,42.7830,194.8337,0.011112,133.2177,278.3840,16.010370,10588
2011-09-30 04:30:28,42.7843,193.5393,0.011082,135.6714,17.3939,16.012015,17686
...,...,...,...,...,...,...,...
2018-04-01 00:04:22,42.7428,200.6065,0.000747,347.8126,12.9725,16.400048,21062
2018-04-01 10:17:36,42.7386,197.7481,0.000672,339.4173,21.5137,16.435739,36794
2018-04-01 11:44:52,42.7393,197.3406,0.000620,338.9313,21.1427,16.442018,5236
2018-04-01 14:39:44,42.7382,196.5219,0.000705,335.9816,24.1336,16.456765,10492


In [3]:
Tiangong.describe()

,Inclination,RAAN,Eccentricity,AoP,Mean Anomaly,Mean Motion,Interval
count,7137.000000,7137.000000,7137.000000,7137.000000,7137.000000,7137.000000,7137.000000
mean,42.771766,179.011506,0.001134,197.154871,178.262322,15.720837,28763.037130
std,0.105683,103.896526,0.000583,104.934894,98.897923,0.105134,15173.329651
min,42.284000,0.029500,0.000030,0.048300,0.012900,15.543259,0.000000
25%,42.762900,88.830500,0.000730,107.436200,96.696900,15.659676,18687.000000
50%,42.769600,178.558500,0.001104,212.601500,179.315700,15.704140,28265.000000
75%,42.780100,269.072200,0.001533,287.775200,259.445000,15.740221,34628.000000
max,51.643800,359.989400,0.011559,359.881500,359.931800,16.465606,224155.000000


In [4]:
values = Tiangong.values
values = values.astype('float32')

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
Tiangong = scaler.fit_transform(values)

In [6]:
Tiangong.shape

(7137, 7)

In [7]:
def series_to_supervised(data, n_in, n_out, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [8]:
n_timesteps = 10 # Number of previous time steps
n_pred = 1 # Number of time steps to predict
Tiangong_reframed = series_to_supervised(Tiangong, n_timesteps, n_pred)

In [9]:
Tiangong_reframed

,var1(t-10),var2(t-10),var3(t-10),var4(t-10),var5(t-10),var6(t-10),var7(t-10),var1(t-9),var2(t-9),var3(t-9),...,var5(t-1),var6(t-1),var7(t-1),var1(t),var2(t),var3(t),var4(t),var5(t),var6(t),var7(t)
10,0.053014,0.545750,0.876018,0.362695,0.640313,0.530657,0.000000,0.052726,0.544628,1.000000,...,0.103913,0.223951,0.245714,0.053484,0.509783,0.115829,0.143122,0.158724,0.223904,0.050113
11,0.052726,0.544628,1.000000,0.365798,0.638708,0.505938,0.024068,0.053388,0.543351,0.952104,...,0.158724,0.223904,0.050113,0.053494,0.504310,0.113261,0.149751,0.255581,0.224236,0.124744
12,0.053388,0.543351,0.952104,0.367825,0.811025,0.505547,0.028239,0.053313,0.541183,0.961237,...,0.255581,0.224236,0.124744,0.053506,0.498912,0.113131,0.156137,0.281068,0.224619,0.122991
13,0.053313,0.541183,0.961237,0.370086,0.773427,0.506439,0.047235,0.053452,0.537587,0.958583,...,0.281068,0.224619,0.122991,0.053516,0.494811,0.113539,0.160931,0.099229,0.224932,0.093440
14,0.053452,0.537587,0.958583,0.376905,0.048291,0.508221,0.078901,0.053442,0.536557,0.958930,...,0.099229,0.224932,0.093440,0.053452,0.492718,0.112767,0.163973,0.048657,0.225082,0.047721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7132,0.049382,0.587712,0.077448,0.970435,0.030699,0.852491,0.094323,0.049382,0.584333,0.073726,...,0.004792,0.909853,0.046994,0.049018,0.557220,0.062190,0.966460,0.036007,0.928925,0.093962
7133,0.049382,0.584333,0.073726,0.974333,0.025956,0.856541,0.070697,0.049168,0.579827,0.086260,...,0.036007,0.928925,0.093962,0.048570,0.549280,0.055668,0.943129,0.059738,0.967621,0.164145
7134,0.049168,0.579827,0.086260,0.956250,0.044188,0.868010,0.094234,0.049168,0.579827,0.086260,...,0.059738,0.967621,0.164145,0.048644,0.548147,0.051218,0.941778,0.058707,0.974428,0.023359
7135,0.049168,0.579827,0.086260,0.956251,0.044497,0.868021,0.000004,0.048003,0.575309,0.064368,...,0.058707,0.974428,0.023359,0.048527,0.545873,0.058548,0.933581,0.067017,0.990416,0.046807


In [10]:
values = Tiangong_reframed.values
values = values.astype('float32')

Tiangong_reframed.drop(Tiangong_reframed.columns[[-1, -3, -4, -5, -6, -7]], axis = 1, inplace = True)

In [11]:
Tiangong_reframed

,var1(t-10),var2(t-10),var3(t-10),var4(t-10),var5(t-10),var6(t-10),var7(t-10),var1(t-9),var2(t-9),var3(t-9),...,var6(t-2),var7(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var6(t)
10,0.053014,0.545750,0.876018,0.362695,0.640313,0.530657,0.000000,0.052726,0.544628,1.000000,...,0.223143,0.072825,0.053869,0.511967,0.107615,0.147698,0.103913,0.223951,0.245714,0.223904
11,0.052726,0.544628,1.000000,0.365798,0.638708,0.505938,0.024068,0.053388,0.543351,0.952104,...,0.223951,0.245714,0.053484,0.509783,0.115829,0.143122,0.158724,0.223904,0.050113,0.224236
12,0.053388,0.543351,0.952104,0.367825,0.811025,0.505547,0.028239,0.053313,0.541183,0.961237,...,0.223904,0.050113,0.053494,0.504310,0.113261,0.149751,0.255581,0.224236,0.124744,0.224619
13,0.053313,0.541183,0.961237,0.370086,0.773427,0.506439,0.047235,0.053452,0.537587,0.958583,...,0.224236,0.124744,0.053506,0.498912,0.113131,0.156137,0.281068,0.224619,0.122991,0.224932
14,0.053452,0.537587,0.958583,0.376905,0.048291,0.508221,0.078901,0.053442,0.536557,0.958930,...,0.224619,0.122991,0.053516,0.494811,0.113539,0.160931,0.099229,0.224932,0.093440,0.225082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7132,0.049382,0.587712,0.077448,0.970435,0.030699,0.852491,0.094323,0.049382,0.584333,0.073726,...,0.904705,0.023497,0.049446,0.561750,0.061948,0.995645,0.004792,0.909853,0.046994,0.928925
7133,0.049382,0.584333,0.073726,0.974333,0.025956,0.856541,0.070697,0.049168,0.579827,0.086260,...,0.909853,0.046994,0.049018,0.557220,0.062190,0.966460,0.036007,0.928925,0.093962,0.967621
7134,0.049168,0.579827,0.086260,0.956250,0.044188,0.868010,0.094234,0.049168,0.579827,0.086260,...,0.928925,0.093962,0.048570,0.549280,0.055668,0.943129,0.059738,0.967621,0.164145,0.974428
7135,0.049168,0.579827,0.086260,0.956251,0.044497,0.868021,0.000004,0.048003,0.575309,0.064368,...,0.967621,0.164145,0.048644,0.548147,0.051218,0.941778,0.058707,0.974428,0.023359,0.990416


In [12]:
values = Tiangong_reframed.values

train_size = int(len(Tiangong_reframed) * 0.8)
# test_size = len(Tiangong_reframed) - train_size

train = values[:train_size, :]
test = values[train_size:, :]

n_obs = n_timesteps * 7
X_train, y_train = train[:, :n_obs], train[:, n_obs:]
X_test, y_test = test[:, :n_obs], test[:, n_obs:]

X_train = X_train.reshape((X_train.shape[0], n_timesteps, 7))
X_test = X_test.reshape((X_test.shape[0], n_timesteps, 7))

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(5701, 10, 7) (5701, 1) (1426, 10, 7) (1426, 1)


In [13]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN
from tensorflow.keras.layers import Input, TimeDistributed, Bidirectional, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


In [14]:
batch_size = 4
lr = 0.01
epoch = 100

In [15]:
K.clear_session()
i = Input(shape = (X_train.shape[1], X_train.shape[2]))

x = LSTM(50, return_sequences = True, activation = tf.nn.relu)(i)

x = LSTM(50, return_sequences = True, activation = tf.nn.relu)(x)

x = LSTM(50, return_sequences = True, activation = tf.nn.relu)(x)

x = LSTM(100, return_sequences = True, activation = tf.nn.relu)(x)

x = LSTM(100, activation = tf.nn.relu)(x)

x = Dense(100)(x)
x = Dense(10)(x)
o = Dense(1)(x)

model = Model(inputs = i, outputs = o)
model.compile(loss = 'mse', optimizer = 'adam')

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 7)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 10, 50)            11600     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 50)            20200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 100)           60400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100 

In [17]:
es = EarlyStopping(monitor = 'val_loss', patience = 15, verbose = 1, mode = 'min')

In [18]:
r = model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size, 
              validation_data = (X_test, y_test), verbose = 1, callbacks = [es])

Epoch 1/100
1426/1426 [==============================] - 56s 39ms/step - loss: 0.0011 - val_loss: 0.0042
Epoch 2/100
1426/1426 [==============================] - 48s 33ms/step - loss: 3.1907e-04 - val_loss: 0.0021
Epoch 3/100
1426/1426 [==============================] - 48s 33ms/step - loss: 2.5233e-04 - val_loss: 0.0014
Epoch 4/100
1426/1426 [==============================] - 41s 29ms/step - loss: 1.9433e-04 - val_loss: 0.0094
Epoch 5/100
1426/1426 [==============================] - 40s 28ms/step - loss: 1.2272e-04 - val_loss: 0.0282
Epoch 6/100
1426/1426 [==============================] - 50s 35ms/step - loss: 8.8138e-05 - val_loss: 0.0020
Epoch 7/100
1426/1426 [==============================] - 37s 26ms/step - loss: 7.5995e-05 - val_loss: 0.0082
Epoch 8/100
1426/1426 [==============================] - 58s 41ms/step - loss: 6.3815e-05 - val_loss: 0.0088
Epoch 9/100
1426/1426 [==============================] - 62s 44ms/step - loss: 5.8401e-05 - val_loss: 0.0077
Epoch 10/100
1426/1426 

In [19]:
plt.plot(r.history['loss'], label = 'train')
plt.plot(r.history['val_loss'], label = 'test')
plt.title('MSE Loss and Val_loss')
plt.legend()
plt.show()

In [20]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size = 4)
print(loss_and_metrics)

357/357 [==============================] - 3s 8ms/step - loss: 0.0075
0.007489917334169149


In [21]:
yhat = model.predict(X_test)
yhat2 = model.predict(X_train)

In [22]:
inv_yhat = np.concatenate((X_test[:, -1, -7:-2], yhat, X_test[:, -1, -1, None]), axis = 1)
inv_yhat2 = np.concatenate((X_train[:, -1, -7:-2], yhat2, X_train[:, -1, -1, None]), axis = 1)

In [23]:
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat2 = scaler.inverse_transform(inv_yhat2)

In [24]:
inv_yhat = inv_yhat[:, -2]
inv_yhat2 = inv_yhat2[:, -2]

In [25]:
inv_y = np.concatenate((X_test[:, -1, -7:-2], y_test, X_test[:, -1, -1, None]), axis = 1)
inv_y2 = np.concatenate((X_train[:, -1, -7:-2], y_train, X_train[:, -1, -1, None]), axis = 1)

In [26]:
inv_y = scaler.inverse_transform(inv_y)
inv_y2 = scaler.inverse_transform(inv_y2)

In [27]:
inv_y = inv_y[:, -2]
inv_y2 = inv_y2[:, -2]

In [28]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse1 = sqrt(mean_squared_error(inv_y, inv_yhat))
print("Test RMSE: %.5f" % rmse1)

rmse2 = sqrt(mean_squared_error(inv_y2, inv_yhat2))
print("Train RMSE: %.5f" % rmse2)

Test RMSE: 0.07982
Train RMSE: 0.00428


In [29]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [30]:
mean_absolute_percentage_error(inv_y, inv_yhat)

0.18900365103036165